<a href="https://colab.research.google.com/github/zi2p/M-machine-learning/blob/main/M33001_%D0%9F%D0%B8%D1%81%D0%B0%D1%80%D0%B5%D0%B2%D0%B0_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

In [ ]:
transactions=pd.read_csv("gdrive/MyDrive/transactions.csv", nrows=1000000) # используем только первые 1000000 строк
tr_mcc_codes=pd.read_csv("gdrive/MyDrive/tr_mcc_codes.csv", sep = ';')
tr_types=pd.read_csv("gdrive/MyDrive/tr_types.csv", sep = ';')
gender_train=pd.read_csv("gdrive/MyDrive/gender_train.csv")

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [ ]:
tr_type_transactions=transactions["tr_type"].sample(1000)                                     # №1.1

transactions_ATM_POS=(tr_types["tr_description"]
                      .str.contains('POS')) | (tr_types["tr_description"]
                                               .str.contains('АТМ'))

types_ATM_POS=tr_types["tr_type"][transactions_ATM_POS]
count_ATM_POS=tr_type_transactions.isin(types_ATM_POS).value_counts()[True]

print(count_ATM_POS/1000)                                                                     # №1.2

0.604


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [ ]:
frequency_type_transactions=transactions["tr_type"].value_counts()    # №2.1

top_10_type_transactions=transactions["tr_type"].value_counts().sort_values(ascending=False).head(10)
top_10_transactions=tr_types[tr_types["tr_type"].isin(top_10_type_transactions.index)]

print(top_10_transactions)                                            # №2.2

     tr_type                                     tr_description
39      7010       Взнос наличных через АТМ (в своем тер.банке)
47      7030  Перевод на карту (с карты) через АТМ (в предел...
59      7070  Перевод на карту (с карты) через Мобильный бан...
60      7071  Перевод на карту (с карты) через Мобильный бан...
98      1010                              Покупка. POS ТУ СБ РФ
99      1030                     Оплата услуги. Банкоматы СБ РФ
100     1100                                Покупка. ТУ  Россия
101     1110                             Покупка. POS ТУ Россия
106     2010              Выдача наличных в АТМ Сбербанк России
129     2370  Списание с карты на карту по операции <перевод...


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [ ]:
customers_amount=transactions[["customer_id","amount"]]

customers_sum_max=customers_amount[customers_amount["amount"]>0].groupby(["customer_id"])["amount"].sum()
customer_max_summ=customers_sum_max.sort_values(ascending=False).head(1)
print("Max суммой приходов на карту: ", customer_max_summ, "\n")                                                   # №3.1

customers_sum_min=customers_amount[customers_amount["amount"] < 0].groupby(["customer_id"])["amount"].sum()
customer_min_summ=customers_sum_min.sort_values(ascending=True).head(1)
print("Max суммой расходов на карте: ", customer_min_summ, "\n")                                                   # №3.2


print("Модуль разницы сумм: ", abs(abs(customer_min_summ.min().item())-abs(customers_sum_max.max().item())))       # №3.3

Max суммой приходов на карту:  customer_id
70780820    1.248115e+09
Name: amount, dtype: float64 

Max суммой расходов на карте:  customer_id
70780820   -1.249952e+09
Name: amount, dtype: float64 

Модуль разницы сумм:  1837317.980000019


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [ ]:
# надо сделать список типов транзакций из топа 10 и найти их в общих транзакциях
top_10_transactions_types=top_10_transactions["tr_type"]
list_top_10_transactions_types=list(top_10_transactions_types)                                                 # список подходящих
in_top_10_transactions=transactions[transactions["tr_type"].isin(list_top_10_transactions_types)]              # подходящие из всего списка

*Мне немного было не понятно задание 4.1 и я решила найти медиану и ср арифметическое как по каждому типу в отдельности так и по всем типам из топ 10 в сумме.*

In [ ]:
# №4.1
arithmetic_mean_amount_by_top_10_types=in_top_10_transactions.groupby("tr_type")["amount"].mean()
median_amount_by_top_10_types=in_top_10_transactions.groupby("tr_type")["amount"].median()


all_arithmetic_mean_amount_by_top_10_types=in_top_10_transactions.groupby("tr_type")["amount"].sum().mean()
all_median_amount_by_top_10_types=in_top_10_transactions.groupby("tr_type")["amount"].sum().median()

print("по каждому из типов: \n\n", "Ср.арифметическое: ", round(arithmetic_mean_amount_by_top_10_types, 2), "\n\n", "Медиана: ", round(median_amount_by_top_10_types, 2), "\n")

print("по всем типам: \n\n", "Ср.арифметическое: ", round(all_arithmetic_mean_amount_by_top_10_types, 2), "\n", "Медиана: ", round(all_median_amount_by_top_10_types, 2), "\n")

по каждому из типов: 

 Ср.арифметическое:  tr_type
1010    -19784.75
1030     -5320.98
1100    -44061.83
1110    -32119.33
2010   -136077.63
2370   -205418.25
7010    276391.79
7030     86104.33
7070     65569.83
7071     66806.83
Name: amount, dtype: float64 

 Медиана:  tr_type
1010     -7411.52
1030     -2245.92
1100    -10188.26
1110    -11207.57
2010    -44918.32
2370    -44918.32
7010    112295.79
7030     13951.52
7070     11319.42
7071      3593.47
Name: amount, dtype: float64 

по всем типам: 

 Ср.арифметическое:  -2003771266.09 
 Медиана:  -657482230.3 



*Аналогично предыдущему пункту нахожу для каждого типа в отдельности и в сумме.*

In [ ]:
# №4.2
customer_id_max=customer_max_summ.index
customer_id_min=customer_min_summ.index
tr_customer=transactions[transactions['customer_id'].isin((customer_id_max, customer_id_min))].groupby(['tr_type'])['amount']
tr_customer_summ=transactions[transactions['customer_id'].isin((customer_id_max, customer_id_min))].groupby(['tr_type'])['amount'].sum()

In [ ]:
print("по каждому из типов: \n\n", "Ср.арифметическое: ", round(tr_customer.mean(), 2), "\n\n", "Медиана: ", round(tr_customer.median(), 2), "\n")
print("по всем типам: \n\n", "Ср.арифметическое: ", round(tr_customer_summ.mean(), 2), "\n", "Медиана: ", round(tr_customer_summ.median(), 2), "\n")

по каждому из типов: 

 Ср.арифметическое:  tr_type
2010   -2941055.87
2330   -2382398.34
2370   -2218504.99
6110      17562.93
7020      14651.29
7021      28500.67
7030      14061.96
7031      19210.36
7034      14306.48
7040      21538.33
7070      13848.21
7071      18600.57
7074      22811.59
Name: amount, dtype: float64 

 Медиана:  tr_type
2010   -3368873.66
2330   -2245915.77
2370   -2245915.77
6110      10028.01
7020      10319.98
7021      28500.67
7030       8848.91
7031      12992.62
7034      14306.48
7040      14396.32
7070       8714.15
7071      12543.44
7074       9387.93
Name: amount, dtype: float64 

по всем типам: 

 Ср.арифметическое:  -141332.15 
 Медиана:  296550.71 



## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [ ]:
expenditure=transactions[transactions["amount"]<0].groupby(["gender"])["amount"]    # траты
entrance=transactions[transactions["amount"]>0].groupby(["gender"])["amount"]       # поступления

In [ ]:
print("Модуль разницы ср. трат по гендеру: ", round(abs(expenditure.mean().get(0)-expenditure.mean().get(1)), 2))   # №5.1
print("Модуль разницы ср. поступлений по гендеру: ", round(abs(entrance.mean().get(0)-entrance.mean().get(1)), 2))  # №5.2

Модуль разницы ср. трат по гендеру:  32718.05
Модуль разницы ср. поступлений по гендеру:  63366.57


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [ ]:
# №6.1
max_income=transactions[transactions["amount"]>0].groupby(["gender","tr_type"])["amount"].sum()
top_10_max_income_0=max_income.get(0).sort_values(ascending=True).head(10)
top_10_max_income_1=max_income.get(1).sort_values(ascending=True).head(10)

In [ ]:
print("Гендер 0: \n", top_10_max_income_0, "\n")
print("Гендер 1: \n", top_10_max_income_1)

Гендер 0: 
 tr_type
4051     1122.96
4210     2245.92
4110     2245.92
2370     2335.75
7075     6737.75
2110    22459.16
1010    31532.66
4100    40334.88
2210    68315.82
1210    83644.87
Name: amount, dtype: float64 

Гендер 1: 
 tr_type
2020      2245.92
4110      4491.84
1010     35317.03
8100     64682.37
6000     79280.83
7041     87590.72
2370    113194.15
7015    121279.45
2110    179673.26
2010    224591.59
Name: amount, dtype: float64


In [ ]:
# №6.2
types_tr_gender_0=list(top_10_max_income_0.index)
types_tr_gender_1=list(top_10_max_income_1.index)

print(list(set(types_tr_gender_0) & set(types_tr_gender_1)))

[2110, 2370, 1010, 4110]


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [ ]:
# №7.1
summ_expenditure=transactions[transactions["amount"]<0].groupby(["gender","mcc_code"])["amount"].sum()
summ_expenditure

gender  mcc_code
0.0     742          -386207.68
        1711         -674897.69
        1799         -261626.73
        2741          -20746.42
        3000       -74431645.27
                       ...     
1.0     8999       -28643346.63
        9222         -988205.17
        9311         -373271.22
        9399        -1810820.67
        9402           -4581.67
Name: amount, Length: 349, dtype: float64

In [ ]:
# №7.2
top_10_summ_expenditure_abs=abs(summ_expenditure.get(0)-summ_expenditure.get(1)).sort_values(ascending=False).head(10)
top_10_summ_expenditure_abs_with_mcc=tr_mcc_codes[tr_mcc_codes["mcc_code"].isin(top_10_summ_expenditure_abs.index)]
print("Топ 10 категорий: \n", top_10_summ_expenditure_abs_with_mcc)

Топ 10 категорий: 
      mcc_code                                mcc_description,,,,
22       4829                              Денежные переводы,,,,
62       5511  Легковой и грузовой транспорт — продажа,сервис...
64       5533                      Автозапчасти и аксессуары,,,,
65       5541                        Станции техобслуживания,,,,
75       5661                               Обувные магазины,,,,
90       5812           Места общественного питания,рестораны,,,
116      5977                             Магазины косметики,,,,
123      6010  Финансовые институты — снятие наличности вручн...
124      6011  Финансовые институты — снятие наличности автом...
162      7995                   Транзакции по азартным играм,,,,


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 

In [ ]:
# №8.1
transactions[["tr_day", "tr_time"]]=transactions["tr_datetime"].str.split(" ",expand=True)
transactions[["tr_hour","tr_minute","tr_second"]]=transactions["tr_time"].str.split(":",expand=True)

In [ ]:
# №8.2
transactions_night=transactions[((transactions["amount"]<0)&(transactions["tr_hour"].astype(int)<=6))].groupby("gender")["amount"].count()
transactions_night

gender
0.0    43510
1.0    46694
Name: amount, dtype: int64

# Дополнительное задание на защиту



В задании 3 выведите id клиента и его пол. Найдите среди других клиентов этого же пола "похожего" клиента. Похожесть измеряется списком категорий, в которых есть траты и есть пополнения. Для сравнения каждой пары клиентов будем считать отношение количества совпадающих категорий к сумме категорий эти двух клиентов.

In [ ]:
customers=transactions[["customer_id","gender","tr_type","amount"]]
customers=customers.groupby(["customer_id","gender","tr_type"]).count()
customers_similarity=customers.groupby(["gender","tr_type"]).count()

In [ ]:
customers_similarity

amount
gender tr_type        
0.0    1010        528
       1030        535
       1100        250
       1110        504
       1200        154
...                ...
1.0    7070        340
       7071        130
       7074         20
       7075          4
       8100          1

[123 rows x 1 columns]

In [ ]:
customers

amount
customer_id gender tr_type        
115617      1.0    1010         14
                   1030          5
                   1110         32
                   2010         40
                   2020          2
...                            ...
99999680    1.0    7010         13
                   7020          1
                   7030         11
                   7031          3
                   7070         22

[12654 rows x 1 columns]

**Сравнение 1 покупателя со всеми его пола на типам:**


In [ ]:
customers["dd"]=customers.reset_index()["amount"]/customers_similarity[0]

In [ ]:
customers

amount
customer_id gender tr_type        
115617      1.0    1010         14
                   1030          5
                   1110         32
                   2010         40
                   2020          2
...                            ...
99999680    1.0    7010         13
                   7020          1
                   7030         11
                   7031          3
                   7070         22

[12654 rows x 1 columns]